In [1]:
from flask import Flask, jsonify, request

app = Flask(__name__)

class GerenciadorTarefas:
    """Gerencia a coleção de tarefas em memória."""
    def __init__(self):
        self.tarefas = []
        self.contador_id = 1
    
    def adicionar_tarefa(self, descricao):
        tarefa = {
            'id': self.contador_id,
            'descricao': descricao,
            'concluida': False
        }
        self.tarefas.append(tarefa)
        self.contador_id += 1
        return tarefa

    def obter_tarefa_por_id(self, tarefa_id):
        return next((t for t in self.tarefas if t['id'] == tarefa_id), None)
    
    def atualizar_tarefa(self, tarefa_id, data):
        tarefa = self.obter_tarefa_por_id(tarefa_id)
        if tarefa:
            if 'descricao' in data:
                tarefa['descricao'] = data['descricao']
            if 'concluida' in data:
                tarefa['concluida'] = data['concluida']
            return tarefa
        return None

    def deletar_tarefa(self, tarefa_id):
        tarefa_inicial = len(self.tarefas)
        self.tarefas = [t for t in self.tarefas if t['id'] != tarefa_id]
        return len(self.tarefas) != tarefa_inicial # Retorna True se a tarefa foi removida

gerenciador = GerenciadorTarefas()

# --- Rotas da API (CRUD) ---

@app.route('/tarefas', methods=['GET'])
def obter_tarefas():
    """Retorna todas as tarefas."""
    return jsonify(gerenciador.tarefas)

@app.route('/tarefas', methods=['POST'])
def criar_tarefa():
    """Cria uma nova tarefa. Requer JSON: {'descricao': '...'}"""
    if not request.json or 'descricao' not in request.json:
        return jsonify({'erro': 'Descricao é obrigatória'}), 400
        
    descricao = request.json.get('descricao')
    tarefa = gerenciador.adicionar_tarefa(descricao)
    return jsonify(tarefa), 201

@app.route('/tarefas/<int:tarefa_id>', methods=['GET'])
def obter_tarefa(tarefa_id):
    """Obtém uma tarefa específica pelo ID."""
    tarefa = gerenciador.obter_tarefa_por_id(tarefa_id)
    if not tarefa:
        return jsonify({'erro': 'Tarefa não encontrada'}), 404
    return jsonify(tarefa)


@app.route('/tarefas/<int:tarefa_id>', methods=['PUT'])
def atualizar_tarefa(tarefa_id):
    """Atualiza uma tarefa existente. Requer JSON: {'descricao': '...' e/ou 'concluida': True/False}"""
    if not request.json:
        return jsonify({'erro': 'Dados de atualização ausentes'}), 400
        
    tarefa_atualizada = gerenciador.atualizar_tarefa(tarefa_id, request.json)
    if not tarefa_atualizada:
        return jsonify({'erro': 'Tarefa não encontrada'}), 404
        
    return jsonify(tarefa_atualizada)

@app.route('/tarefas/<int:tarefa_id>', methods=['DELETE'])
def deletar_tarefa(tarefa_id):
    """Deleta uma tarefa pelo ID."""
    if gerenciador.deletar_tarefa(tarefa_id):
        return jsonify({'mensagem': f'Tarefa {tarefa_id} deletada com sucesso'}), 204 # No Content
    return jsonify({'erro': 'Tarefa não encontrada'}), 404

if __name__ == '__main__':
    # Para rodar localmente: flask run
    # Adicionando algumas tarefas iniciais para teste
    gerenciador.adicionar_tarefa("Configurar o ambiente Flask")
    gerenciador.adicionar_tarefa("Implementar rotas GET e POST")
    app.run(debug=True)

ModuleNotFoundError: No module named 'flask'